In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error


/Users/abohane/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# cleaned datasets with weather data

train_data = pd.read_csv('df_train.csv')
test_data = pd.read_csv('df_test.csv')

train_data.set_index('date', inplace=True)
test_data.set_index('date', inplace=True)

In [3]:
# Step 2: Define target variables and features
target_variables = ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25']

# Features available in both train and test sets
common_features = ['temperature', 'humidite', 'vent_moyen', 'vent_direction', 'pluie_1h']

# Time features (assuming they are derived from 'date' and can be computed for the test set)
time_features = ['hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos', 'month_sin', 'month_cos']

# Lag features and rolling statistics (not available in test set, we'll handle them separately)
lag_features = [col for col in train_data.columns if 'lag' in col]
rolling_features = [col for col in train_data.columns if 'rolling' in col]

# Interaction features (we can recompute them)
interaction_features = ['NO2_CO_interaction', 'PM10_PM25_interaction']

# All features for training
feature_columns = common_features + time_features + lag_features + rolling_features + interaction_features


In [4]:
# Step 3: Prepare the data for modeling

# Ensure the date index is in datetime format
train_data.index = pd.to_datetime(train_data.index)
test_data.index = pd.to_datetime(test_data.index)

# Generate time features for test_data
test_data['hour'] = test_data.index.hour
test_data['day_of_week'] = test_data.index.dayofweek
test_data['month'] = test_data.index.month

# Create cyclic features
test_data['hour_sin'] = np.sin(2 * np.pi * test_data['hour'] / 24)
test_data['hour_cos'] = np.cos(2 * np.pi * test_data['hour'] / 24)
test_data['day_of_week_sin'] = np.sin(2 * np.pi * test_data['day_of_week'] / 7)
test_data['day_of_week_cos'] = np.cos(2 * np.pi * test_data['day_of_week'] / 7)
test_data['month_sin'] = np.sin(2 * np.pi * test_data['month'] / 12)
test_data['month_cos'] = np.cos(2 * np.pi * test_data['month'] / 12)

# Drop intermediate columns
test_data.drop(['hour', 'day_of_week', 'month'], axis=1, inplace=True)

In [5]:
# Select the final features for modeling
final_features = common_features + time_features + interaction_features


In [6]:
# Handle missing values in train_data
train_data = train_data.dropna()


In [7]:
# Step 4: Implement time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)

In [8]:
# Step 5 (Adjusted): Train separate models for each pollutant

# Prepare the training data
X = train_data[final_features]

# Initialize a dictionary to store models for each pollutant
models = {}
mae_scores = []

# Iterate through each split
for fold, (train_index, val_index) in enumerate(tscv.split(X)):
    print(f"Training fold {fold + 1}...")
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]

    fold_mae = {}
    fold_models = {}

    for pollutant in target_variables:
        print(f"  Training model for {pollutant}...")
        y_train = train_data.iloc[train_index][pollutant]
        y_val = train_data.iloc[val_index][pollutant]

        # Initialize the LightGBM regressor
        lgbm_reg = lgb.LGBMRegressor(
            objective='regression_l1',
            n_estimators=1000,
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42
        )

        # Train the model
        lgbm_reg.fit(
            X_train,
            y_train,
            eval_set=[(X_val, y_val)],
            eval_metric='mae',
        )

        # Predict on validation set
        y_pred = lgbm_reg.predict(X_val)

        # Compute MAE
        mae = mean_absolute_error(y_val, y_pred)
        fold_mae[pollutant] = mae
        fold_models[pollutant] = lgbm_reg

    mae_scores.append(fold_mae)

    # After last fold, save the models
    if fold == tscv.get_n_splits() - 1:
        models = fold_models  # Use models from the last fold



Training fold 1...
  Training model for valeur_NO2...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000250 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1636
[LightGBM] [Info] Number of data points in the train set: 6808, number of used features: 13
[LightGBM] [Info] Start training from score 16.200001
  Training model for valeur_CO...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000124 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1636
[LightGBM] [Info] Number of data points in the train set: 6808, number of used features: 13
[LightGBM] [Info] Start training from score 0.175000
  Training model for valeur_O3...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000374 seconds.
You can set `force_col_wise

In [9]:
# Analyze MAE scores
mae_df = pd.DataFrame(mae_scores)
print("MAE scores across folds:")
print(mae_df)
print("\nAverage MAE:")
print(mae_df.mean())

MAE scores across folds:
   valeur_NO2  valeur_CO  valeur_O3  valeur_PM10  valeur_PM25
0    7.340618   0.082819  10.965511     2.715187     1.513925
1    5.607390   0.053429   8.098471     1.808041     1.089944
2    4.417616   0.053574   8.792999     1.705617     0.899640
3    2.657229   0.029609   7.674783     1.644027     0.902760
4    2.407709   0.026069   7.523596     1.208033     0.672109

Average MAE:
valeur_NO2     4.486112
valeur_CO      0.049100
valeur_O3      8.611072
valeur_PM10    1.816181
valeur_PM25    1.015676
dtype: float64


In [10]:
# Step 6: Implement recursive forecasting for the test set

# Initialize test_data with necessary features
test_data_pred = test_data.copy()
test_data_pred[interaction_features] = 0  # Initialize interaction features

# We need to generate lag features and rolling statistics recursively
# We'll start by getting the last known values from the training data

# Get the last known values for lag features
last_known_values = train_data.iloc[-168:]  # Get last 168 hours for lag168

# Initialize an empty DataFrame to store predictions
test_predictions = pd.DataFrame(index=test_data_pred.index, columns=target_variables)

# Iterate over each time step in the test set
for i, timestamp in enumerate(test_data_pred.index):
    # Prepare the input features
    X_test = test_data_pred.loc[[timestamp], common_features + time_features + interaction_features]

    # For the first few steps, we can use the last known lag values from the training set
    if i == 0:
        # Lag 1
        for pollutant in target_variables:
            lag1_value = train_data.iloc[-1][pollutant]
            X_test[f'{pollutant}_lag1'] = lag1_value
        # Lag 24
        if len(train_data) >= 24:
            for pollutant in target_variables:
                lag24_value = train_data.iloc[-24][pollutant]
                X_test[f'{pollutant}_lag24'] = lag24_value
        else:
            # Not enough data for lag24, use last known value
            for pollutant in target_variables:
                X_test[f'{pollutant}_lag24'] = train_data.iloc[-1][pollutant]
        # Lag 168
        if len(train_data) >= 168:
            for pollutant in target_variables:
                lag168_value = train_data.iloc[-168][pollutant]
                X_test[f'{pollutant}_lag168'] = lag168_value
        else:
            # Not enough data for lag168, use last known value
            for pollutant in target_variables:
                X_test[f'{pollutant}_lag168'] = train_data.iloc[-1][pollutant]
        # Rolling means and stds
        for pollutant in target_variables:
            rolling_mean24 = train_data[pollutant].rolling(window=24).mean().iloc[-1]
            rolling_std24 = train_data[pollutant].rolling(window=24).std().iloc[-1]
            rolling_mean168 = train_data[pollutant].rolling(window=168).mean().iloc[-1]
            rolling_std168 = train_data[pollutant].rolling(window=168).std().iloc[-1]
            X_test[f'{pollutant}_rolling_mean24'] = rolling_mean24
            X_test[f'{pollutant}_rolling_std24'] = rolling_std24
            X_test[f'{pollutant}_rolling_mean168'] = rolling_mean168
            X_test[f'{pollutant}_rolling_std168'] = rolling_std168
    else:
        # Use previous predictions for lag features
        for pollutant in target_variables:
            # Lag 1
            X_test[f'{pollutant}_lag1'] = test_predictions.iloc[i - 1][pollutant]
            # Lag 24
            if i >= 24:
                X_test[f'{pollutant}_lag24'] = test_predictions.iloc[i - 24][pollutant]
            else:
                # Use last known value from training data
                X_test[f'{pollutant}_lag24'] = train_data.iloc[-24 + i][pollutant]
            # Lag 168
            if i >= 168:
                X_test[f'{pollutant}_lag168'] = test_predictions.iloc[i - 168][pollutant]
            else:
                # Use last known value from training data
                X_test[f'{pollutant}_lag168'] = train_data.iloc[-168 + i][pollutant]
            # Rolling means and stds
            # For rolling calculations, we'll need to collect previous predictions
            if i < 24:
                recent_values = pd.concat([train_data[pollutant].iloc[-(24 - i):], test_predictions[pollutant].iloc[:i]])
            else:
                recent_values = test_predictions[pollutant].iloc[i - 24:i]
            rolling_mean24 = recent_values.mean()
            rolling_std24 = recent_values.std()
            if i < 168:
                recent_values_168 = pd.concat([train_data[pollutant].iloc[-(168 - i):], test_predictions[pollutant].iloc[:i]])
            else:
                recent_values_168 = test_predictions[pollutant].iloc[i - 168:i]
            rolling_mean168 = recent_values_168.mean()
            rolling_std168 = recent_values_168.std()
            X_test[f'{pollutant}_rolling_mean24'] = rolling_mean24
            X_test[f'{pollutant}_rolling_std24'] = rolling_std24
            X_test[f'{pollutant}_rolling_mean168'] = rolling_mean168
            X_test[f'{pollutant}_rolling_std168'] = rolling_std168

    # Update interaction features
    X_test['NO2_CO_interaction'] = X_test['valeur_NO2_lag1'] * X_test['valeur_CO_lag1']
    X_test['PM10_PM25_interaction'] = X_test['valeur_PM10_lag1'] * X_test['valeur_PM25_lag1']

    # Ensure all required features are present
    for col in feature_columns:
        if col not in X_test.columns:
            X_test[col] = 0  # or appropriate default value

    # Predict for each pollutant
    for pollutant in target_variables:
        model = models[pollutant]
        prediction = model.predict(X_test[final_features])[0]
        test_predictions.loc[timestamp, pollutant] = prediction

    # Update test_data_pred with the new interaction features (if necessary)
    # test_data_pred.loc[timestamp, 'NO2_CO_interaction'] = X_test['NO2_CO_interaction']
    # test_data_pred.loc[timestamp, 'PM10_PM25_interaction'] = X_test['PM10_PM25_interaction']


In [11]:
test_predictions.head()

,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25
date,,,,,
2024-09-03 23:00:00,20.267291,0.2117,42.797193,8.516558,4.747267
2024-09-04 00:00:00,18.205523,0.217526,44.219245,8.525977,4.689853
2024-09-04 01:00:00,16.606518,0.226198,43.005045,8.299093,4.945912
2024-09-04 02:00:00,16.963767,0.218774,42.879589,8.72156,4.766376
2024-09-04 03:00:00,17.110345,0.210415,42.785725,8.710404,4.664991


In [12]:
test_predictions = test_predictions.reset_index().rename(columns={'date': 'id'})


In [13]:
test_predictions.head()

,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25
0,2024-09-03 23:00:00,20.267291,0.2117,42.797193,8.516558,4.747267
1,2024-09-04 00:00:00,18.205523,0.217526,44.219245,8.525977,4.689853
2,2024-09-04 01:00:00,16.606518,0.226198,43.005045,8.299093,4.945912
3,2024-09-04 02:00:00,16.963767,0.218774,42.879589,8.72156,4.766376
4,2024-09-04 03:00:00,17.110345,0.210415,42.785725,8.710404,4.664991


In [14]:
test_predictions['id'] = pd.to_datetime(test_predictions['id']).dt.strftime('%Y-%m-%d %H')

In [15]:
test_predictions.head()

,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25
0,2024-09-03 23,20.267291,0.2117,42.797193,8.516558,4.747267
1,2024-09-04 00,18.205523,0.217526,44.219245,8.525977,4.689853
2,2024-09-04 01,16.606518,0.226198,43.005045,8.299093,4.945912
3,2024-09-04 02,16.963767,0.218774,42.879589,8.72156,4.766376
4,2024-09-04 03,17.110345,0.210415,42.785725,8.710404,4.664991


In [19]:
test_predictions.to_csv('test_predictions.csv', index=False)